In [1]:
from transformers import AutoTokenizer

#加载分词器
tokenizer = AutoTokenizer.from_pretrained('DPR')
from transformers import AutoModel

#加载预训练模型
pretrained = AutoModel.from_pretrained('DPR')
print(tokenizer)

#分词测试
inputs = tokenizer.batch_encode_plus(
    [[
        '海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间',
        '的', '海', '域', '。'
    ],
     [
         '这', '座', '依', '山', '傍', '水', '的', '博', '物', '馆', '由', '国', '内', '一',
         '流', '的', '设', '计', '师', '主', '持', '设', '计', '，', '整', '个', '建', '筑',
         '群', '精', '美', '而', '恢', '宏', '。'
     ]],
    truncation=True,
    padding=True,
    return_tensors='pt',
    is_split_into_words=True)

print(inputs.keys())
for i in inputs:
    print(i,inputs[i].shape)
pretrained(**inputs).last_hidden_state.shape
#del out3['token_embeddings'],out3['sentence_embedding']
pretrained(**inputs).last_hidden_state.shape

Some weights of BertModel were not initialized from the model checkpoint at DPR and are newly initialized: ['pooler.dense.weight', 'pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


BertTokenizerFast(name_or_path='DPR', vocab_size=23283, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
input_ids torch.Size([2, 37])
token_type_ids torch.Size([2, 37])
attention_mask torch.Size([2, 37])


torch.Size([2, 37, 768])

In [2]:
tokenizer2 = AutoTokenizer.from_pretrained("hfl/chinese-roberta-wwm-ext")
lawformer = AutoModel.from_pretrained("thunlp/Lawformer")
print(tokenizer)

#分词测试
inputs = tokenizer2.batch_encode_plus(
    [[
        '海', '钓', '比', '赛', '地', '点', '在', '厦', '门', '与', '金', '门', '之', '间',
        '的', '海', '域', '。'
    ],
     [
         '这', '座', '依', '山', '傍', '水', '的', '博', '物', '馆', '由', '国', '内', '一',
         '流', '的', '设', '计', '师', '主', '持', '设', '计', '，', '整', '个', '建', '筑',
         '群', '精', '美', '而', '恢', '宏', '。'
     ]],
    truncation=True,
    padding=True,
    return_tensors='pt',
    is_split_into_words=True)

print(inputs.keys())
for i in inputs:
    print(i,inputs[i].shape)
lawformer(**inputs).last_hidden_state.shape
#del out3['token_embeddings'],out3['sentence_embedding']
lawformer(**inputs).last_hidden_state.shape

Some weights of the model checkpoint at thunlp/Lawformer were not used when initializing LongformerModel: ['lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerModel were not initialized from the model checkpoint at thunlp/Lawformer and are newly initialized: ['longformer.pooler.dense.weight', 'longformer.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be ab

BertTokenizerFast(name_or_path='DPR', vocab_size=23283, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
input_ids torch.Size([2, 37])
token_type_ids torch.Size([2, 37])
attention_mask torch.Size([2, 37])


torch.Size([2, 37, 768])

In [8]:
dpr=pretrained
import torch
import json
class CailDataset(torch.utils.data.Dataset):
    def __init__(self, path):
        self.path = path 
        file = open(path,'r',encoding='utf-8')
        cases = file.readlines()
        self.cases = [json.loads(i) for i in cases]
        self.length = len(self.cases)
    def __len__(self):
        return self.length

    def __getitem__(self, index):
        data = self.cases[index]
        caseA = "".join(data['Case_A'])
        caseB = "".join(data['Case_B'])
        label = data['label']
        inputs = tokenizer([caseA,caseB], return_tensors="pt",padding="max_length",max_length=300,truncation=True)
        return inputs,label

def collate_fn(data):
    tensors = [i[0] for i in data]
    labels = [i[1]  for i in data]
    inputs_ids = torch.cat([i['input_ids'] for i in tensors])
    token_type_ids = torch.cat([i['token_type_ids'] for i in tensors])
    mask = torch.cat([i['attention_mask'] for i in tensors])
    inputs = {"input_ids":inputs_ids,"token_type_ids":token_type_ids,"attention_mask":mask}
    return inputs,labels

trainset = CailDataset('./cail/competition_stage_2_train.json')
#数据加载器
trainloader = torch.utils.data.DataLoader(dataset=trainset,
                                     batch_size=1,
                                     collate_fn = collate_fn,
                                     shuffle=True,
                                     drop_last=True)



#模型试算
#[b, lens] -> [b, lens, 768]
#pretrained(**inputs).last_hidden_state.shape
import torch  
class Lawfomer2Super(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.tuneing = False
        self.pretrained = dpr
        # 300 is max length of lawformer 
        self.conv1d = torch.nn.Conv1d(in_channels=300,out_channels=150,kernel_size=1)
        self.conv2d=torch.nn.Conv1d(in_channels=150,out_channels=1,kernel_size=1)
        self.conv3d  = torch.nn.Conv2d(in_channels = 1, out_channels = 1,kernel_size=1)
        self.fc = torch.nn.Linear(768, 3)

    def forward(self, inputs):
        #print(inputs.keys())
        # inputs : batch_size * max_length * 768 
        # conv1d output : batch_size * 1 * 768
        # result : batch_size*1 * 3 
        if self.tuneing:
            out = self.pretrained(**inputs).last_hidden_state
        else:
            with torch.no_grad():
                out = self.pretrained(**inputs).last_hidden_state
        #print("initial shape:",out.shape)
        
        out = self.conv1d(out)
        #print("After conv1d:",out.shape)
        out=self.conv2d(out)
        #print("After conv2d:",out.shape)
        out = out.squeeze()
        #print("After 1 squeeze:",out.shape)
        out = out.unsqueeze(0)
        out=out.unsqueeze(0)
        #print("After 2 unsqueezes:",out.shape)
        out = self.conv3d(out)
        #print("After conv2d:",out.shape)
        out = self.fc(out)
        
        out = out.squeeze()
        out=torch.abs(out[0]-out[1]) #这里把out的两个batch的对应位置相减再取绝对值，是为了尽量突出差距，但效果不佳。
        
        out=torch.tensor([list(out)])  #match crossentropyloss
        #print("Final:",out)
        
        return out

    def fine_tuneing(self, tuneing):
        self.tuneing = tuneing
        if tuneing:
            for i in self.pretrained.parameters():
                i.requires_grad = True

            lawformer.train()
            self.pretrained =dpr
        else:
            for i in dpr.parameters():
                i.requires_grad_(False)

            lawformer.eval()
            #print("any")
            self.pretrained = None

lf2classify = Lawfomer2Super()
lf2classify.fine_tuneing(tuneing=True)
device = "cuda:0"
lf2classify.to(device=device)
lf2classify.pretrained.to(device)
from transformers import AdamW
#训练
from tqdm import tqdm
def train(epochs,device):
    lr = 2e-4 if lf2classify.tuneing else 5e-4

    #训练
    optimizer = AdamW(lf2classify.parameters(), lr=lr)
    criterion = torch.nn.CrossEntropyLoss()

    lf2classify.train()
    for epoch in range(epochs):
        lf2classify.train()
        step = 0
        correct = 0
        for step, (inputs,labels) in tqdm(enumerate(trainloader),total=len(trainloader),colour='green'):
            #print("labels",labels)
            step+=1
            #模型计算
            for i in inputs:
                inputs[i] = inputs[i].to(device)
            outs = lf2classify(inputs)
            outs=outs.to(device)
            #梯度下降
            #outs = outs.squeeze()
            labels = torch.tensor(labels).long()
            #labels=torch.tensor([list(labels)])
            labels = labels.to(device)
            #print("cretirion",outs,labels)
            loss = criterion(outs,labels)
            loss.requires_grad_(True)  #loss requires grad
            fake = torch.argmax(outs)
            
            correct += fake==labels
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        print("in epoch "+str(epoch)+" current train loss is "+str(loss.item()))
        print("accuracy is ",correct/len(trainset.cases))

        torch.save(lf2classify, 'model/dpr_微调.model')

train(5000,device)
print(sum(p.numel() for p in lf2classify.parameters()))

100%|██████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:40<00:00, 73.62it/s]

in epoch 0 current train loss is 1.2100224494934082
accuracy is  tensor([0.3393], device='cuda:0')


FileNotFoundError: [Errno 2] No such file or directory: 'model/dpr_微调.model'

In [43]:
c=torch.nn.CrossEntropyLoss()
x=torch.tensor([0.002,-0.098,0.300])
x=torch.tensor([list(x)])
y=torch.tensor([1])
c(x,y),x,y

(tensor(1.2793), tensor([[ 0.0020, -0.0980,  0.3000]]), tensor([1]))

In [6]:
import torch
import numpy as np
b=torch.tensor([0.01,0.02,0.03])
c=torch.tensor([0.02,0.03,0.01])
a=torch.abs(b-c)
a

tensor([0.0100, 0.0100, 0.0200])